# Grupo 1 - Smog predicition

Inicialmente se va a analizar el dataset proporcionado, buscando que variables pueden ser útiles para realizar el reto.

In [ ]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

df = pd.read_csv('data/train.csv')
df.head()

De un primer vistazo, algunos datos que pueden ser de utilidad para realizar la predicción podrían ser "Engine Size (L)", "Cylinders", "Transmission", "Fuel Type", "Comb (L/100 km)" y "CO2 Emissions (g/km)". 

In [ ]:
df.info()

Hemos decidido dividir la columna "Transmission" para poder obtener el número de marchas ("Gears") y el tipo de transmisión ("Transmission") de cada coche.

In [ ]:
df['Gears'] = df['Transmission'].str.extract('(\d+)')
df['Gears'] = pd.to_numeric(df['Gears'], errors='coerce')
df['Transmission'] = df['Transmission'].str.extract('(\D+)')
df.head()

Es necesario pasar los valores categóricos a numéricos para poder visualizar los datos y entrenar el modelo correctamente.

In [ ]:
#Fuel Type
df.loc[df["Fuel Type"] == "X", "Fuel Type"] = 0
df.loc[df["Fuel Type"] == "Z", "Fuel Type"] = 1
df.loc[df["Fuel Type"] == "D", "Fuel Type"] = 2
df.loc[df["Fuel Type"] == "E", "Fuel Type"] = 3
df.loc[df["Fuel Type"] == "N", "Fuel Type"] = 4

#Transmission
df.loc[df["Transmission"] == "A", "Transmission"] = 0
df.loc[df["Transmission"] == "AM", "Transmission"] = 1
df.loc[df["Transmission"] == "AS", "Transmission"] = 2
df.loc[df["Transmission"] == "AV", "Transmission"] = 3
df.loc[df["Transmission"] == "M", "Transmission"] = 4


#Vehicle Class
df.loc[df["Vehicle Class"] == "Compact", "Vehicle Class"] = 0
df.loc[df["Vehicle Class"] == "Full-size", "Vehicle Class"] = 1
df.loc[df["Vehicle Class"] == "Mid-size", "Vehicle Class"] = 2
df.loc[df["Vehicle Class"] == "Minicompact", "Vehicle Class"] = 3
df.loc[df["Vehicle Class"] == "Minivan", "Vehicle Class"] = 4
df.loc[df["Vehicle Class"] == "Minicompact", "Vehicle Class"] = 5
df.loc[df["Vehicle Class"] == "Pickup truck: Small", "Vehicle Class"] = 6
df.loc[df["Vehicle Class"] == "Pickup truck: Standard", "Vehicle Class"] = 7
df.loc[df["Vehicle Class"] == "SUV: Small", "Vehicle Class"] = 8
df.loc[df["Vehicle Class"] == "SUV: Standard", "Vehicle Class"] = 9
df.loc[df["Vehicle Class"] == "Special purpose vehicle", "Vehicle Class"] = 10
df.loc[df["Vehicle Class"] == "Station wagon: Mid-size", "Vehicle Class"] = 11
df.loc[df["Vehicle Class"] == "Station wagon: Small", "Vehicle Class"] = 12
df.loc[df["Vehicle Class"] == "Subcompact", "Vehicle Class"] = 13
df.loc[df["Vehicle Class"] == "Two-seater", "Vehicle Class"] = 14

In [ ]:
df.isnull().sum()

Al buscar si al tratar los datos hay algun valor nulo encontramos que en la columna "Gears" hay 22 valores nulos. Por ello a los valores nulos se les asocia la media de los valores no nulos. 

In [ ]:

df['Gears'].fillna(df['Gears'].mean(), inplace=True)
df.isnull().sum()

Al analizar el dataset hemos detectado que la columna de "Model Year" es igual en todos los datos, por lo que hemos decidido suprimirla al no aportar gran información para realizar el modelo.

In [ ]:
df.drop("Model Year", axis=1, inplace=True)
df.head()

También, hemos decidido eliminar las columnas de "Make" y "Model" porque no resultan útiles, al no aportar información relevante para calcular la clasificación de "Smog". Además, así podemos realizar un modelo menos complejo al tener menos características a ajustar.

In [ ]:
df.drop("Make", axis=1, inplace=True)
df.drop("Model", axis=1, inplace=True)
df.head()

Al igual, se eliminará la columna "Comb (mpg)" al ser una métrica equivalente a la anterior, pero en unidades del sistema anglosajón.

In [ ]:
df.drop("Comb (mpg)", axis=1, inplace=True)
df.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)

%matplotlib inline

In [ ]:
#g = sns.PairGrid(df, hue="Smog")
#g.map_diag(plt.hist)
#g.map_offdiag(plt.scatter)
#g.add_legend()

Finalmente, eliminamos las columnas "Fuel Consumption City (L/100Km)" y "Hwy (L/100Km)" ya que estos datos se han combinado en la columna "Comb (L/100Km)", que será la que se usará como media del consumo de cada coche en ciudad y autopista en nuestro estudio. 

In [ ]:
df.drop("Fuel Consumption City (L/100 km)", axis=1, inplace=True)
df.drop("Hwy (L/100 km)", axis=1, inplace=True)
df.head()

In [ ]:
g = sns.PairGrid(df, hue="Smog")
g.map_diag(plt.hist)
g.map_offdiag(plt.scatter)
g.add_legend()

Tras observar los gráficos de las diferentes características correladas podemos deducir que las principales variables para la predicción son "Gears", "Fuel Type", "Cylinders" y "Transmission". En cambio, una de las variables que pensábamos que iban a ser de las más prometedoras era "Engine Size (L)", pero al final ha resultado ser de las menos correladas con el resto.